In [0]:
def get_data() -> DataFrame:
    """
    Function to get the most recent Sunday's data snapshot from Snowflake,
    remove quotes from column names, and return as a Spark DataFrame.
    """
    
    # Helper function that returns the previous Sunday relative to today's date
    def get_last_sunday() -> str:
        today = datetime.date.today()
        weekday = today.weekday()
        last_sunday = today - datetime.timedelta(days=weekday+1)
        return last_sunday.strftime('%Y-%m-%d')

    # Helper function to remove quotes from column names
    def remove_quotes_from_columns(df: DataFrame) -> DataFrame:
        old_columns = df.schema.names
        new_columns = [x.replace('"', '') for x in old_columns]
        return df.toDF(*new_columns)

    # Get the most recent Sunday date as a string
    str_date = get_last_sunday()

    # Get secrets for accessing Snowflake (assuming dbutils is available)
    prod_user = dbutils.secrets.get(scope="SMARTY", key="smarty_sc_u")
    prod_pwd = dbutils.secrets.get(scope="SMARTY", key="smarty_sc_p")

    # Snowflake options
    options = {
        "sfUrl": 'threemobile.west-europe.privatelink.snowflakecomputing.com',
        "sfUser": prod_user,
        "sfPassword": prod_pwd,
        "sfDatabase": 'PROD_SMARTY',
        "sfSchema": 'DEFAULT',
        "sfWarehouse": 'PROD_SMARTY_WH',
        "sfRole": 'SF_PROD_SMARTY'
    }
    
    # Load the data for last sunday
    sdf = spark.read \
                .format("snowflake") \
                .options(**options) \
                .option("query",
                        f"""
                        SELECT *
                        FROM SMARTY_CAR
                        WHERE "SnapshotDate" = '{str_date}'
                        """) \
                .load()

    # Remove quotes from column names
    sdf = remove_quotes_from_columns(sdf)

    return sdf, str_date